In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

import zipfile
import os

In [3]:
file = zipfile.ZipFile('data\week20200207.zip')

In [4]:
file.getinfo

<bound method ZipFile.getinfo of <zipfile.ZipFile filename='data\\week20200207.zip' mode='r'>>

In [9]:
temp = file.read(file.filelist[0].filename)

In [17]:
eod_data = temp.decode('utf-8').split('\r\n')
len(eod_data)

1604

In [23]:
splitted_eod_data = [val.split(',') for val in eod_data]

In [28]:
df = pd.DataFrame(splitted_eod_data, columns=['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df.head()

,Code,Date,Open,High,Low,Close,Volume
0,360,20200203,3.28,3.28,3.15,3.17,96249
1,14D,20200203,0.175,0.175,0.17,0.17,135850
2,1AD,20200203,0.105,0.105,0.1,0.1,100000
3,1AG,20200203,0.042,0.042,0.038,0.038,667017
4,1ST,20200203,0.061,0.061,0.058,0.061,169462


In [29]:
df.loc[df['Code'] == 'WES', :]

,Code,Date,Open,High,Low,Close,Volume
1526,WES,20200203,44.8,45.21,44.55,45.09,1325076
